<a href="https://www.kaggle.com/code/chenzhijing3121/finbert-sentiment-analysis?scriptVersionId=267542276" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# ===================== 0) Setup (Kaggle) =====================
!pip -q install -U transformers peft

# ===================== 1) Imports & Env ======================
import os, warnings, numpy as np, pandas as pd, torch
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import numpy as np
import pandas as pd

import transformers
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model

print("transformers==", transformers.__version__)
print("torch==", torch.__version__, "| cuda:", torch.cuda.is_available())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-10-12 16:07:32.925875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760285253.130608      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760285253.191307      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


transformers== 4.57.0
torch== 2.6.0+cu124 | cuda: True


In [2]:
# ===================== 2) Model / Tokeniser ==================
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

MODEL_NAME = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

def build_base_model():
    return AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=3,
        id2label=id2label,
        label2id=label2id,
    )

def build_lora_model():
    base = build_base_model()
    lora_cfg = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=16, lora_alpha=32, lora_dropout=0.05,
        target_modules=["query","value"],  
        bias="none",
    )
    return get_peft_model(base, lora_cfg)

def train_once(model, train_data, eval_data, out_dir, *, lr, bsz_train, bsz_eval, epochs):
    args = TrainingArguments(
        output_dir=out_dir,
        learning_rate=lr,
        per_device_train_batch_size=bsz_train,
        per_device_eval_batch_size=bsz_eval,
        num_train_epochs=epochs,
        lr_scheduler_type="linear",
        optim="adamw_torch",
        logging_steps=100,
        report_to="none",
        seed=42,
        fp16=torch.cuda.is_available(),
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_data,
        eval_dataset=eval_data,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,  
    )
    trainer.train()
    metrics_val = trainer.evaluate(eval_data)
    return trainer, metrics_val



# ===================== 3) Load & Split Data ==================
filename = "/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv"
df = pd.read_csv(
    filename, names=["sentiment", "text"],
    encoding="ISO-8859-1", engine="python"
)

parts_train, parts_test = [], []
for s in ["positive", "neutral", "negative"]:
    tr, te = train_test_split(
        df[df.sentiment == s], train_size=300, test_size=300, random_state=42
    )
    parts_train.append(tr); parts_test.append(te)

X_train = pd.concat(parts_train) 
X_test  = pd.concat(parts_test)

used_idx = set(X_train.index) | set(X_test.index)
X_eval = (df.loc[df.index.difference(used_idx)]
            .groupby('sentiment', group_keys=False)
            .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))

X_train = X_train.sample(frac=1, random_state=10).reset_index(drop=True)
X_test  = X_test.reset_index(drop=True)
X_eval  = X_eval.reset_index(drop=True)

for _df in [X_train, X_test, X_eval]:
    _df["labels"] = _df.sentiment.map(label2id).astype("int64")

# ===================== 4) Build HF Datasets ==================
def tok_fn(batch):
    return tokenizer(
        batch["text"], padding="max_length", truncation=True, max_length=256
    )

train_data = Dataset.from_pandas(X_train).map(tok_fn, batched=True)\
    .remove_columns(["text","sentiment"])
test_data  = Dataset.from_pandas(X_test).map(tok_fn, batched=True)\
    .remove_columns(["text","sentiment"])
eval_data  = Dataset.from_pandas(X_eval).map(tok_fn, batched=True)\
    .remove_columns(["text","sentiment"])

cols = ["input_ids", "attention_mask", "labels"]
train_data.set_format("torch", columns=cols)
test_data.set_format("torch",  columns=cols)
eval_data.set_format("torch",  columns=cols)

# ===================== 5) Metrics ============================
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "f1": f1_score(labels, preds, average="weighted"),
        "accuracy": accuracy_score(labels, preds),
    }

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [3]:
# ===================== Train Full & LoRA =====================
# Full fine-tuning
print("🔹 Training FULL fine-tuning...")
trainer_full, res_full = train_once(
    model=build_base_model(),
    train_data=train_data, eval_data=eval_data,
    out_dir=f"finetuned_full_{MODEL_NAME.split('/')[-1]}",
    lr=2e-5, bsz_train=8, bsz_eval=8, epochs=5
)
print(f"[FULL] val_f1={res_full['eval_f1']:.5f} | val_acc={res_full['eval_accuracy']:.5f}")

# LoRA fine-tuning
print("\n🔹 Training LoRA fine-tuning...")
trainer_lora, res_lora = train_once(
    model=build_lora_model(),
    train_data=train_data, eval_data=eval_data,
    out_dir=f"finetuned_lora_{MODEL_NAME.split('/')[-1]}",
    lr=1e-3, bsz_train=32, bsz_eval=64, epochs=5
)
print(f"[LoRA] val_f1={res_lora['eval_f1']:.5f} | val_acc={res_lora['eval_accuracy']:.5f}")

# ===================== Test & Compare on hold-out test set =====================
def eval_trainer(trainer, test_data, name="MODEL"):
    pred = trainer.predict(test_data)
    y_true = pred.label_ids
    y_pred = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average="macro")
    f1_weighted = f1_score(y_true, y_pred, average="weighted")
    print(f"\n===== {name} : Test Report =====")
    print(classification_report(y_true, y_pred, target_names=[id2label[i] for i in sorted(id2label)] , digits=4))
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred, labels=sorted(id2label.keys())))
    print(f"Accuracy:    {acc:.4f}")
    print(f"F1 (macro):  {f1_macro:.4f}")
    print(f"F1 (weighted): {f1_weighted:.4f}")
    return {"name": name, "accuracy": acc, "f1_macro": f1_macro, "f1_weighted": f1_weighted}

m_full = eval_trainer(trainer_full, test_data, name="FinBERT Full FT")
m_lora = eval_trainer(trainer_lora, test_data, name="FinBERT LoRA")

cmp_df = pd.DataFrame([m_full, m_lora]).set_index("name")
print("\n===== Summary (Test) =====")
print(cmp_df.sort_values("f1_macro", ascending=False))



🔹 Training FULL fine-tuning...


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Step,Training Loss
100,1.014200
200,0.303900
300,0.124400
400,0.070600
500,0.037500


[FULL] val_f1=0.89767 | val_acc=0.90000

🔹 Training LoRA fine-tuning...


Step,Training Loss
100,0.318600


[LoRA] val_f1=0.91185 | val_acc=0.91333

===== FinBERT Full FT : Test Report =====
              precision    recall  f1-score   support

    negative     0.9262    0.9200    0.9231       300
     neutral     0.8694    0.7767    0.8204       300
    positive     0.8024    0.8933    0.8454       300

    accuracy                         0.8633       900
   macro avg     0.8660    0.8633    0.8630       900
weighted avg     0.8660    0.8633    0.8630       900

Confusion matrix:
[[276  13  11]
 [ 12 233  55]
 [ 10  22 268]]
Accuracy:    0.8633
F1 (macro):  0.8630
F1 (weighted): 0.8630

===== FinBERT LoRA : Test Report =====
              precision    recall  f1-score   support

    negative     0.9412    0.9600    0.9505       300
     neutral     0.8912    0.8467    0.8684       300
    positive     0.8673    0.8933    0.8801       300

    accuracy                         0.9000       900
   macro avg     0.8999    0.9000    0.8997       900
weighted avg     0.8999    0.9000    0.8997 